# Running Tune experiments with Optuna

This example demonstrates the usage of Optuna with Ray Tune via `OptunaSearch`, including conditional search spaces and multi-objective use cases.

While you may use a scheduler with `OptunaSearch`, e.g. `AsyncHyperBandScheduler`, please note that schedulers may not work correctly with multi-objective optimization, since they often expect a scalar score.

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [1]:
import time
from typing import Dict, Optional, Any

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch

Toggle `use_func` and `multi_obj` below to test different example usages.

In [2]:
# Whether to use function to define search space
use_func = False
# Whether to run multi-objective
multi_obj = False

Let's start by defining a simple evaluation function.
The form here is explicit for demonstration, yet it is typically a black-box.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune some hyperparameters,
namely `width`, `height`, and `activation`.

In [3]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [4]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)
              
def multi_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluate(step, config["width"], config["height"], config["activation"])
        # Feed the score back back to Tune.
        tune.report(
           iterations=step, loss=intermediate_score, gain=intermediate_score * width
        )

In [5]:
ray.init(configure_logging=False)

E0323 09:28:50.950964100     135 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1648052930.950951700","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}


{'node_ip_address': '192.168.1.181',
 'raylet_ip_address': '192.168.1.181',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-23_09-28-45_356778_135/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-23_09-28-45_356778_135/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-23_09-28-45_356778_135',
 'metrics_export_port': 61005,
 'gcs_address': '192.168.1.181:59996',
 'address': '192.168.1.181:59996',
 'node_id': '4ece46b4ef07f28b63daa19ca5bcc8637c3e7b2dc54c20ec7a352c2a'}

Next we define a search space. The critical assumption is that the optimal hyperparamters live within this space. Yet, if the space is very large, then those hyperparamters may be difficult to find in a short amount of time.

The search space can be defined as independent or conditional. The simplest case is where we have an independent search space. In this case, a config dictionary will suffice.

In [6]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

However, for the conditional search space, we must use a define-by-run function.

In [7]:
def define_by_run_func(trial) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``tune.run`` (in this example, that's
    ``objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
    /tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """

    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("width", 0, 20)
        trial.suggest_float("height", -100, 100) 
    else:
        trial.suggest_float("width", 0, 21)
        trial.suggest_float("height", -101, 101)
        
    # Return all constants in a dictionary.
    return {"steps": 100}

Optionally, we may provide an initial set of hyperparameters that we believe as the current best, (`current_best_params`), and
pass this belief into a `OptunaSearch` searcher.
Furthermore, we optionally define a `scheduler` to go along with our algorithm.

We then set the maximum concurrent trials to `4` with a `ConcurrencyLimiter`.
Lastly, we must set the number of samples for this Tune run to `1000`
(you can decrease this if it takes too long on your machine).

In [8]:
initial_params = [
    {"width": 1, "height": 2, "activation": "relu"},
    {"width": 4, "height": 2, "activation": "relu"},
]

# Choosing specified case:
if not multi_obj: 
    if not use_func:
        # Initial guess
        searcher = OptunaSearch(points_to_evaluate=initial_params)
    else:
        # Search space from function, initial guess
        searcher = OptunaSearch(space=define_by_run_func, points_to_evaluate=initial_params)
else: 
    if not use_func:
        # Initial guess, multi-objective
        searcher = OptunaSearch(points_to_evaluate=initial_params, metric=["loss", "gain"], mode=["min", "max"])
    else:
        # Search space from function, initial guess, multi-objective
        searcher = OptunaSearch(space=define_by_run_func, points_to_evaluate=initial_params, metric=["loss", "gain"], mode=["min", "max"])

algo = ConcurrencyLimiter(searcher, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()

num_samples = 1000

In [9]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is running the experiment.

In [10]:
# Single objective case
if not multi_obj:
    analysis = tune.run(
        objective,
        search_alg=algo,
        scheduler=scheduler,
        metric="mean_loss",
        mode="min",
        num_samples=num_samples,
        config=search_space if not use_func else None
    )

    print("Best hyperparameters found were: ", analysis.best_config)

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


[I 2022-03-23 09:28:51,197] A new study created in memory with name: optuna


/mnt/c/Users/brett/anyscale/venv/lib/python3.8/site-packages/ray/tune/suggest/optuna.py:361: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
  self._ot_study.enqueue_trial(point)
/mnt/c/Users/brett/anyscale/venv/lib/python3.8/site-packages/optuna/study/study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
/mnt/c/Users/brett/anyscale/venv/lib/python3.8/site-packages/optuna/study/study.py:856: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  self.add_trial(


Trial name,status,loc,activation,height,width
objective_54daa288,PENDING,,relu,2,1


Result for objective_54daa288:
  date: 2022-03-23_09-28-55
  done: false
  experiment_id: 91316a345035432bb55f100249b0e6c5
  hostname: brett
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 20.2
  neg_mean_loss: -20.2
  node_ip: 192.168.1.181
  pid: 255
  time_since_restore: 0.10064482688903809
  time_this_iter_s: 0.10064482688903809
  time_total_s: 0.10064482688903809
  timestamp: 1648052935
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 54daa288
  
Result for objective_5542b346:
  date: 2022-03-23_09-28-56
  done: false
  experiment_id: 60ed273299fc4d5e9183710926cc8fb0
  hostname: brett
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 20.2
  neg_mean_loss: -20.2
  node_ip: 192.168.1.181
  pid: 254
  time_since_restore: 0.10046768188476562
  time_this_iter_s: 0.10046768188476562
  time_total_s: 0.10046768188476562
  timestamp: 1648052936
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5542b346
  


...

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_5d3f401e,RUNNING,192.168.1.181:540,tanh,-43.6083,2.65515,-3.71994,56,5.9881,55,3.71994
objective_54daa288,TERMINATED,192.168.1.181:255,relu,2,1,11.1174,100,10.6812,99,-11.1174
objective_5542b346,TERMINATED,192.168.1.181:254,relu,2,4,10.4463,100,10.7478,99,-10.4463
objective_555a431c,TERMINATED,192.168.1.181:256,relu,50.7516,2.58579,25.0752,1,0.100636,0,-25.0752
objective_55727e46,TERMINATED,192.168.1.181:253,relu,-66.5353,13.8334,3.41896,100,10.6568,99,-3.41896
objective_55e26d00,TERMINATED,192.168.1.181:252,relu,6.88503,1.20451,20.6885,1,0.101324,0,-20.6885
objective_564fe876,TERMINATED,192.168.1.181:251,tanh,1.11576,17.2121,0.16992,100,10.6501,99,-0.16992
objective_5c013888,TERMINATED,192.168.1.181:257,tanh,24.5378,13.2826,4.45988,4,0.412206,3,-4.45988
objective_5c211b8a,TERMINATED,192.168.1.181:248,relu,69.0573,3.74972,26.9057,1,0.10051,0,-26.9057
objective_5c3e220c,TERMINATED,192.168.1.181:541,relu,-80.5409,6.34018,5.39175,4,0.426356,3,-5.39175


Result for objective_5d3f401e:
  date: 2022-03-23_09-29-19
  done: false
  experiment_id: 858717afde094bf68163659990482e08
  hostname: brett
  iterations: 95
  iterations_since_restore: 96
  mean_loss: -3.9795003728384075
  neg_mean_loss: 3.9795003728384075
  node_ip: 192.168.1.181
  pid: 540
  time_since_restore: 10.233474016189575
  time_this_iter_s: 0.10848188400268555
  time_total_s: 10.233474016189575
  timestamp: 1648052959
  timesteps_since_restore: 0
  training_iteration: 96
  trial_id: 5d3f401e
  


Result for objective_5d3f401e:
  date: 2022-03-23_09-29-19
  done: true
  experiment_id: 858717afde094bf68163659990482e08
  hostname: brett
  iterations: 99
  iterations_since_restore: 100
  mean_loss: -3.9943430303597025
  neg_mean_loss: 3.9943430303597025
  node_ip: 192.168.1.181
  pid: 540
  time_since_restore: 10.649172306060791
  time_this_iter_s: 0.10361504554748535
  time_total_s: 10.649172306060791
  timestamp: 1648052959
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: 5d3f401e
  


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_54daa288,TERMINATED,192.168.1.181:255,relu,2,1,11.1174,100,10.6812,99,-11.1174
objective_5542b346,TERMINATED,192.168.1.181:254,relu,2,4,10.4463,100,10.7478,99,-10.4463
objective_555a431c,TERMINATED,192.168.1.181:256,relu,50.7516,2.58579,25.0752,1,0.100636,0,-25.0752
objective_55727e46,TERMINATED,192.168.1.181:253,relu,-66.5353,13.8334,3.41896,100,10.6568,99,-3.41896
objective_55e26d00,TERMINATED,192.168.1.181:252,relu,6.88503,1.20451,20.6885,1,0.101324,0,-20.6885
objective_564fe876,TERMINATED,192.168.1.181:251,tanh,1.11576,17.2121,0.16992,100,10.6501,99,-0.16992
objective_5c013888,TERMINATED,192.168.1.181:257,tanh,24.5378,13.2826,4.45988,4,0.412206,3,-4.45988
objective_5c211b8a,TERMINATED,192.168.1.181:248,relu,69.0573,3.74972,26.9057,1,0.10051,0,-26.9057
objective_5c3e220c,TERMINATED,192.168.1.181:541,relu,-80.5409,6.34018,5.39175,4,0.426356,3,-5.39175
objective_5d3f401e,TERMINATED,192.168.1.181:540,tanh,-43.6083,2.65515,-3.99434,100,10.6492,99,3.99434


Best hyperparameters found were:  {'steps': 100, 'width': 2.6551503672837007, 'height': -43.608314920153155, 'activation': 'tanh'}


In [11]:
# Multi-objective case: removed scheduler. metric, mode passed through "search_alg"
if multi_obj:
    analysis = tune.run(
        multi_objective,
        search_alg=algo,
        num_samples=num_samples,
        config=search_space if not use_func else None
    )
    print("Best hyperparameters for loss found were: ", analysis.get_best_config("loss", "min"))
    print("Best hyperparameters for gain found were: ", analysis.get_best_config("gain", "max"))

In [12]:
ray.shutdown()